In [52]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [51]:
file_path = '/Users/yarden/Documents/Atuda/spring_2024/causal_inference/Causal-Inference-Project-Effect-of-Age-on-Graduating/code/data/processed_data.csv'
data = pd.read_csv(file_path)

data.head()

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Mother's qualification,Father's qualification,Mother's occupation,...,Admission grade,Displaced,Educational special needs,Gender,Scholarship holder,Unemployment rate,Inflation rate,GDP,Target,Adult
0,1,17,5,171,1,Complete Secondary Education,122.0,Incomplete Secondary Education,Complete Secondary Education,Administrative,...,127.3,1,0,1,0,10.8,1.4,1.74,0,0
1,1,15,1,9254,1,Complete Secondary Education,160.0,Complete Secondary Education,Higher Education - Undergraduate,Professionals,...,142.5,1,0,1,0,13.9,-0.3,0.79,1,0
2,1,1,5,9070,1,Complete Secondary Education,122.0,Other Specific Qualifications,Other Specific Qualifications,Plant and Machine Operators,...,124.8,1,0,1,0,10.8,1.4,1.74,0,0
3,1,17,2,9773,1,Complete Secondary Education,122.0,Other Specific Qualifications,Other Specific Qualifications,Administrative,...,119.6,1,0,0,0,9.4,-0.8,-3.12,1,0
4,2,39,1,8014,0,Complete Secondary Education,100.0,Other Specific Qualifications,Other Specific Qualifications,Plant and Machine Operators,...,141.5,0,0,0,0,13.9,-0.3,0.79,1,1


In [58]:
n = len(data)

t = data['Adult']
y = data['Target']

X = data.drop(columns=['Adult', 'Target'])

numerical_columns = ['Previous qualification (grade)', 'Admission grade', 'Unemployment rate', 'Inflation rate', 'GDP']
categorical_columns = list(set(X.columns) - set(numerical_columns))

# Scaling numerical columns
scaler = StandardScaler()
X[numerical_columns] = scaler.fit_transform(X[numerical_columns])

# One-hot encoding for categorical columns
X = pd.get_dummies(X, columns=categorical_columns, drop_first=True)

# Covariate Adjustment

## S-learner

In [59]:
# Combine features and treatment variable to create the full feature set
X_full = pd.concat([X, t], axis=1)

# Define the target variable (binary target)
y = data['Target']


# Fit a Random Forest Classifier model to predict the target variable
model = RandomForestClassifier(random_state=42)
model.fit(X_full, y)    

# Make predictions for all data points
y_pred_all = model.predict(X_full)

In [60]:
# Filter predictions where Adult = 1 and Adult = 0
treated_predictions_sum = sum(y_pred_all[data['Adult'] == 1])
control_predictions_sum = sum(y_pred_all[data['Adult'] == 0])

# Calculate the desired value
ATE = (treated_predictions_sum - control_predictions_sum) / n
ATT = treated_predictions_sum / sum(data['Adult'])
ATC = control_predictions_sum / (n - sum(data['Adult']))

print(f'ATE: {ATE}')
print(f'ATT: {ATT}')
print(f'ATC: {ATC}')

ATE: -0.20169451635678984
ATT: 0.3598420755781162
ATC: 0.6037964458804523


## T-learner

In [63]:
X_1 = X[X_full['Adult'] == 1]
X_0 = X[X_full['Adult'] == 0]

y_1 = data[data['Adult'] == 1]['Target']
y_0 = data[data['Adult'] == 0]['Target']

# Fit a Random Forest Classifier model to predict the target variable for each subgroup
model_1 = RandomForestClassifier(random_state=42)
model_1.fit(X_1, y_1)

model_0 = RandomForestClassifier(random_state=42)
model_0.fit(X_0, y_0)

y_pred_all_1 = model_1.predict(X)
y_pred_all_0 = model_0.predict(X)

In [64]:
treated_predictions_sum = sum(y_pred_all_1)
control_predictions_sum = sum(y_pred_all_0)

ATE = (treated_predictions_sum - control_predictions_sum) / n
ATT = (sum(y_pred_all_1[data['Adult'] == 1]) -  sum(y_pred_all_0[data['Adult'] == 1])) / sum(data['Adult'])
ATC = (sum(y_pred_all_1[data['Adult'] == 0]) -  sum(y_pred_all_0[data['Adult'] == 0])) / (n - sum(data['Adult']))
                                 
print(f'ATE: {ATE}')
print(f'ATT: {ATT}')
print(f'ATC: {ATC}')

ATE: -0.1536832195810779
ATT: -0.15792442188381275
ATC: -0.15064620355411953


# IPW

In [67]:
# Fit a logistic regression model to estimate the propensity score (probability of treatment)
propensity_model = LogisticRegression(random_state=42)
propensity_model.fit(X, t)

# Calculate the propensity scores (predicted probabilities of receiving the treatment)
propensity_scores = propensity_model.predict_proba(X)[:, 1]

# Add the propensity scores to the original data
data['propensity_score'] = propensity_scores

/Users/yarden/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [68]:
ATE = sum(data['Target'] * data['Adult'] / data['propensity_score']) / n - sum(data['Target'] * (1 - data['Adult']) / (1 - data['propensity_score'])) / n
ATT = sum(data['Target'] * data['Adult']) / sum(data['Adult']) - sum(data['Target'] * (1 - data['Adult']) * data['propensity_score'] / (1 - data['propensity_score'])) / sum((1 - data['Adult']) * data['propensity_score'] / (1 - data['propensity_score']))
p = sum(data['Adult']) / n
ATC = (ATE - p * ATT) / (1 - p)

print(f'ATE: {ATE}')
print(f'ATT: {ATT}')
print(f'ATC: {ATC}')

ATE: 0.019604997537647217
ATT: -0.053394143603745114
ATC: 0.07187780740989626


# Propensity Score Matching

In [76]:
treated = data[data['Adult'] == 1]
control = data[data['Adult'] == 0]

In [77]:
def nearest_neighbor_match(treated, control, treatment_outcome, control_outcome):
    matched_outcomes = []
    
    # Loop over each treated individual
    for i, treated_row in treated.iterrows():
        # Find the control individual with the closest propensity score
        control_row = control.iloc[(control['propensity_score'] - treated_row['propensity_score']).abs().argsort()[:1]]
        
        # Calculate the difference in outcomes
        matched_outcomes.append(treated_row[treatment_outcome] - control_row[control_outcome].values[0])
    
    return matched_outcomes

In [78]:
att_differences = nearest_neighbor_match(treated, control, 'Target', 'Target')
ATT = sum(att_differences) / len(att_differences)

atc_differences = nearest_neighbor_match(control, treated, 'Target', 'Target')
ATC = sum(atc_differences) / len(atc_differences)

p = sum(data['Adult']) / n
ATE = p * ATT + (1 - p) * ATC

print(f'ATE: {ATE}')
print(f'ATT: {ATT}')
print(f'ATC: {ATC}')

ATE: 0.07813603200753118
ATT: 0.16130851663846588
ATC: 0.018578352180936994


# Doubly Robust Estimator

In [72]:
X_1 = X[X_full['Adult'] == 1]
X_0 = X[X_full['Adult'] == 0]

y_1 = data[data['Adult'] == 1]['Target']
y_0 = data[data['Adult'] == 0]['Target']

# Fit a Random Forest Classifier model to predict the target variable for each subgroup
model_1 = RandomForestClassifier(random_state=42)
model_1.fit(X_1, y_1)

model_0 = RandomForestClassifier(random_state=42)
model_0.fit(X_0, y_0)

y_pred_all_1 = model_1.predict(X)
y_pred_all_0 = model_0.predict(X)

g_1_score = y_pred_all_1 + (data['Adult'] / data['propensity_score']) * (data['Target'] - y_pred_all_1)
g_0_score = y_pred_all_0 + ((1 - data['Adult']) / (1 - data['propensity_score'])) * (data['Target'] - y_pred_all_0)

data['g_1_score'] = g_1_score
data['g_0_score'] = g_0_score

In [49]:
ATE = sum(data['g_1_score']) / n - sum(data['g_0_score']) / n

print(f'ATE: {ATE}')

ATE: -0.1905269041450876
